## Assignment 3

### 1. Implementation of the EM-Algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = np.loadtxt(open("2d-em.csv", "rb"), delimiter=",", skiprows=1)

In [ ]:
class EM():
    def __init__(self, X, number_of_clusters, iterations, show_plot=False):
        self.X = X
        self.K = number_of_clusters
        self.iterations = iterations
        self.show_plot = show_plot
        self.mu = None
        self.pi = None
        self.cov = None
        self.result = None

    def __initialize_parameters(self):
#         update this part to randomly generate parameters
        self.mu = [[2,2], [-2,-2]]
        self.cov = [[[1,0],[0,1]], [[1,0],[0,1]]]
        self.pi = [0.5, 0.5]
    
    def __multivar_gauss(self, x, mu, sigma):
        return 1 / ( ((2*np.pi)**(len(mu)/2)) * (np.linalg.det(sigma)**(1/2)) ) * np.exp((-1/2) * ((x-mu).T.dot(np.linalg.inv(sigma))).dot((x-mu)))
    
    def __predict_cluster(self, r):
        return r.argmax(axis=1)
    
    def __show_plot(self, r):
        fig2 = plt.figure(figsize=(5,5))
        ax2 = fig2.add_subplot(111)
        ax2.scatter(self.X[:,0], self.X[:,1], c=self.__predict_cluster(r), s=1)
    
    def run(self):
        """Plot the initial state"""   
        fig1 = plt.figure(figsize=(5,5))
        ax1 = fig1.add_subplot(111)
        ax1.scatter(self.X[:,0],self.X[:,1], s=1)
        ax1.set_title('Initial state')

        self.__initialize_parameters()
    
        for i in range(self.iterations):
            """expectation step"""
            r = np.zeros((len(self.X), self.K))
            for c in range(self.K):
                r[:,c] = [self.pi[c]*self.__multivar_gauss(x, self.mu[c], self.cov[c]) for x in self.X]
            for j in range(len(r)):
                r[j] = r[j]/(np.sum(self.pi)*np.sum(r,axis=1)[j])
            
            """maximisation step"""
            self.mu = []
            self.cov = []
            self.pi = []
            for c in range(self.K):
                m_c = np.sum(r[:,c],axis=0)
                mu_c = (1/m_c)*np.sum(self.X*r[:,c].reshape(len(self.X),1),axis=0)
                self.mu.append(mu_c)
                self.cov.append(((1/m_c)*np.dot((np.array(r[:,c]).reshape(len(self.X),1)*(self.X-mu_c)).T,(self.X-mu_c))))
                self.pi.append(m_c/np.sum(r))

            if self.show_plot and i%5 == 0:
                self.__show_plot(r)
        
        self.result = r
        plt.show()
    
    def plot(self):
        clusters = self.__predict_cluster(self.result)
        fig3 = plt.figure(figsize=(10,10))
        ax3 = fig3.add_subplot(111)
        ax3.scatter(self.X[:,0], self.X[:,1], c=clusters, s=1)
        
        centers = np.array([np.mean(self.X[np.where(clusters == i)], axis=0) for i in range(self.K)])
        ax3.scatter(centers[:,0], centers[:,1],  marker='x', s=200, linewidths=2)

        ax3.set_title('Final state')
        plt.show()

In [ ]:
em = EM(data, 2 , 10, show_plot=False)
em.run()

In [ ]:
em.plot()